In [8]:
#!/usr/bin/env python
################################
# General setup                #
################################

import os
import sys
import pandas as pd
import pickle
import numpy as np
sys.path.insert(0, '../../LIB')
from metrics import check_for_spikes
from libMVO import fix_trace_id
sys.path.insert(0, './AAA-master/automatic_processing')
from config import Config
from features import FeatureVector
from tools import extract_features
import obspy
import datetime

### PREPARE THE CATALOG DataFrame ###
SEISAN_DATA = os.path.join( os.getenv('HOME'),'DATA','MVO') # e.g. /home/user/seismo
pandaSeisDir = os.path.join(SEISAN_DATA, 'miniseed_c') # e.g. /home/user/seismo/pandaSeis
SEISAN_DB = 'MVOE_' # e.g. the seisan database name (e.g. MVOE_)
PROJECTDIR = os.path.join(os.getenv('HOME'),'src', 'kitchensinkGT', 'PROJECTS', 'MontserratML') # this dir
csvfile_external = os.path.join(SEISAN_DATA, 'MachineLearning', SEISAN_DB, 'runAAA', 'MVOE_11_labelled_events.csv')
csvfile_internal = 'catalog/30_MVO_labelled_events_filtered.csv' # has to match that in AAA-master/config/general/newsettings_10.json
csvfile_internal = './AAA-master/MONTSERRAT/' + csvfile_internal
output_path_cat = csvfile_internal.replace('.csv', '.pd')
alltraces_file = '30_alltraceDFs.csv'
add_GT_metrics = True
metrics_to_add = ['bandratio_[0.8_4.0_16.0]', 'bandratio_[1.0_6.0_11.0]', \
                  'bw_max', 'bw_min', 'kurtosis', 'skewness', 'peakF', 'medianF']

# Change if you want your screen to keep quiet
# 0 = quiet
# 1 = in between
# 2 = detailed information
verbatim = 1

# Init project with configuration file
config = Config('./AAA-master/config/general/newsettings_10.json', verbatim=verbatim)
config.readAndCheck()  
cat = pd.read_csv(csvfile_internal)

# Get or define usefull stuff
features = FeatureVector(config, verbatim=verbatim)

# Save featuresList to pickle file
if not os.path.isdir('features'):
    os.makedirs('features')
        
# Glenn. path has miniseed_c hardcoded at start. I want to change this to whatever the config says
WAVTOPDIR = config.data_to_analyze['path_to_data'] 

# read catalog 
for i in range(len(cat.index)):
    if verbatim > 1:
        print('Processing waveform %d of %d' % (i, catalog_length))
    secondFloat = cat.iloc[i]['second']
    tStartSignature = datetime.datetime(int(cat.iloc[i]['year']),     \
                                        int(cat.iloc[i]['month']),    \
                                        int(cat.iloc[i]['day']),      \
                                        int(cat.iloc[i]['hour']),     \
                                        int(cat.iloc[i]['minute']),   \
                                        int(secondFloat), \
                                        int((secondFloat-int(secondFloat))*1000000)) #microseconds
    duration = cat.iloc[i]['length']
    mseedpath = cat.iloc[i]['path']     
    mseedpath = mseedpath.replace('miniseed_c', WAVTOPDIR)
    mseedbase = os.path.basename(mseedpath)
    
    # check is MSEED file exists. copy it to local if do not have already. read it.
    mseedlocal = mseedpath.replace('miniseed_c', 'miniseed_c_local')
    if not os.path.isfile(mseedpath):        
        print("File not found: ",mseedpath)
        continue
    else:
        mseedlocaldir = os.path.dirname(mseedlocal)
        if not os.path.isdir(mseedlocaldir):
            os.makedirs(mseedlocaldir)
        os.system('cp %s %s' % (mseedpath, mseedlocal))
        
        print('Reading ',mseedpath)
        st = obspy.read(mseedpath)
        
    ###### SPIKES CHECK - won't be needed when we reprocess all data #####
    # The reason we do the spike check on the raw WAV file is 
    # because filters run to produce the MSEED file distort the spike
    wavpath = mseedpath.replace('miniseed_c', 'WAV').replace('.mseed','')
    if os.path.isfile(wavpath):
        rawst = obspy.read(wavpath)
        for tr in rawst:
            check_for_spikes(tr)

        good_traces = 0
        trace_ids_to_eliminate = []
        fix_trace_id(rawst)
        for tr in rawst:
            check_for_spikes(tr)
            if tr.stats.quality_factor > 1.0:
                good_traces += 1
            else:
                trace_ids_to_eliminate.append(tr.id)

        for tr in st:
            if tr.id in trace_ids_to_eliminate:
                st.remove(tr)
    ################ END OF SPIKES CHECK ########################
    if len(st)==0:
        continue
        
    if add_GT_metrics:
        tracecsv = mseedpath.replace('.mseed','.csv')
        if os.path.isfile(tracecsv):
            print('Reading ',tracecsv)
            tracedf = pd.read_csv(tracecsv)
        else:
            tracecsv2 = os.path.join(os.getenv('HOME'),'DATA', 'MVO', tracecsv)
            if os.path.isfile(tracecsv2):
                print('Reading ',tracecsv2)
                tracedf = pd.read_csv(tracecsv2)
                os.system('cp %s %s' % (tracecsv2,tracecsv))
            else:
                print(tracecsv, ' not found')
    
    for tr in st:
        print('Processing ', tr.id)
        featurespkl = os.path.join('features',mseedbase.replace('.mseed', '.%s.pkl' % tr.id)) 
        if os.path.exists(featurespkl):
            print(featurespkl, ' exists. Skipping')
            continue
        
        # Get information about recording
        fs = tr.stats['sampling_rate']         
        length_n = tr.stats['npts'] # only change from read_ubinas
        
        if fs < 70.0 or length_n < 1000:
            continue         

        # Preprocessing & features extraction
        y = [tr.data]
        featuresList = extract_features(config, y, features, fs)
        
        # Optionally add precomputed features from Seisan2Pandas
        if add_GT_metrics:
            thistracedf = tracedf[tracedf['id']==tr.id]
            for col in metrics_to_add:
                featuresList = np.append(featuresList, thistracedf[col])
            bandwidth = thistracedf['bw_max'] - thistracedf['bw_min']
            featuresList = np.append(featuresList, bandwidth)
        
        print('Writing ',featurespkl)
        with open(featurespkl, 'wb') as f:
            pickle.dump(featuresList, f)

./AAA-master/config/specific/usecase1_continuous_classification/usecase1_EXAMPLE.json
Welcome to this automatic analysis architecture
Copyright: Marielle MALFANTE - GIPSA-Lab
Univ. Grenoble Alpes, CNRS, Grenoble INP, GIPSA-lab, 38000 Grenoble, France

 *** PROJECT CONFIGURATION 10 ***  
Configuration object from <path> ./AAA-master/config/general/newsettings_10.json, <configuration_number> 10,
 <general> {'project_root': './AAA-master/', 'analysis_type': 'continuous', 'path_to_specific_settings_file': 'config/specific/usecase1_continuous_classification/usecase1_EXAMPLE.json', 'path_to_res': 'res/', 'path_to_visuals': 'fig/', 'path_to_res_to_review': 'res_to_review/'},
 <application> {'name': 'montserrat'},
 <preprocessing> {'energy_norm': True},
 <learning> {'algo': RandomForestClassifier(criterion='entropy'), 'cv': StratifiedShuffleSplit(n_splits=50, random_state=None, test_size=0.5,
            train_size=0.5), 'path_to_catalogue': 'catalog/30_MVO_labelled_events_filtered.pd'},
 <fea

Writing  features/9610-25-0028-06S.MVO_21_1.MV.MBGH..BHN.pkl
Processing  MV.MBGH..BHE
Writing  features/9610-25-0028-06S.MVO_21_1.MV.MBGH..BHE.pkl
Processing  MV.MBWH..SHZ
Writing  features/9610-25-0028-06S.MVO_21_1.MV.MBWH..SHZ.pkl
Processing  MV.MBWH..SNZ
Writing  features/9610-25-0028-06S.MVO_21_1.MV.MBWH..SNZ.pkl
Processing  MV.MBBE..BHZ
Writing  features/9610-25-0028-06S.MVO_21_1.MV.MBBE..BHZ.pkl
Processing  MV.MBBE..BHN
Writing  features/9610-25-0028-06S.MVO_21_1.MV.MBBE..BHN.pkl
Processing  MV.MBBE..BHE
Writing  features/9610-25-0028-06S.MVO_21_1.MV.MBBE..BHE.pkl
Processing  MV.MBGB..BHZ
Writing  features/9610-25-0028-06S.MVO_21_1.MV.MBGB..BHZ.pkl
Processing  MV.MBGB..BHN
Writing  features/9610-25-0028-06S.MVO_21_1.MV.MBGB..BHN.pkl
Processing  MV.MBGB..BHE
Writing  features/9610-25-0028-06S.MVO_21_1.MV.MBGB..BHE.pkl
Reading  ./miniseed_c/MVOE_/1996/10/9610-25-0753-29S.MVO_21_1.mseed
Reading  /Users/thompsong/DATA/MVO/./miniseed_c/MVOE_/1996/10/9610-25-0753-29S.MVO_21_1.csv
Proce

Writing  features/9610-26-0255-47S.MVO_21_1.MV.MBGB..BHN.pkl
Processing  MV.MBGB..BHE
Writing  features/9610-26-0255-47S.MVO_21_1.MV.MBGB..BHE.pkl
Reading  ./miniseed_c/MVOE_/1996/10/9610-26-0533-53S.MVO_21_1.mseed
Reading  /Users/thompsong/DATA/MVO/./miniseed_c/MVOE_/1996/10/9610-26-0533-53S.MVO_21_1.csv
Processing  MV.MBGA..BHZ
Writing  features/9610-26-0533-53S.MVO_21_1.MV.MBGA..BHZ.pkl
Processing  MV.MBGA..BHN
Writing  features/9610-26-0533-53S.MVO_21_1.MV.MBGA..BHN.pkl
Processing  MV.MBGA..BHE
Writing  features/9610-26-0533-53S.MVO_21_1.MV.MBGA..BHE.pkl
Processing  MV.MBLG..SHZ
Writing  features/9610-26-0533-53S.MVO_21_1.MV.MBLG..SHZ.pkl
Processing  MV.MBLG..SNZ
Writing  features/9610-26-0533-53S.MVO_21_1.MV.MBLG..SNZ.pkl
Processing  MV.MBRY..SNZ
Writing  features/9610-26-0533-53S.MVO_21_1.MV.MBRY..SNZ.pkl
Processing  MV.MBGE..BHZ
Writing  features/9610-26-0533-53S.MVO_21_1.MV.MBGE..BHZ.pkl
Processing  MV.MBGE..BHN
Writing  features/9610-26-0533-53S.MVO_21_1.MV.MBGE..BHN.pkl
Proce

Writing  features/9610-27-0523-04S.MVO_21_1.MV.MBLG..SNZ.pkl
Processing  MV.MBRY..SNZ
Writing  features/9610-27-0523-04S.MVO_21_1.MV.MBRY..SNZ.pkl
Processing  MV.MBGE..BHZ
Writing  features/9610-27-0523-04S.MVO_21_1.MV.MBGE..BHZ.pkl
Processing  MV.MBGE..BHN
Writing  features/9610-27-0523-04S.MVO_21_1.MV.MBGE..BHN.pkl
Processing  MV.MBGE..BHE
Writing  features/9610-27-0523-04S.MVO_21_1.MV.MBGE..BHE.pkl
Processing  MV.MBGH..BHZ
Writing  features/9610-27-0523-04S.MVO_21_1.MV.MBGH..BHZ.pkl
Processing  MV.MBGH..BHN
Writing  features/9610-27-0523-04S.MVO_21_1.MV.MBGH..BHN.pkl
Processing  MV.MBGH..BHE
Writing  features/9610-27-0523-04S.MVO_21_1.MV.MBGH..BHE.pkl
Processing  MV.MBWH..SHZ
Writing  features/9610-27-0523-04S.MVO_21_1.MV.MBWH..SHZ.pkl
Processing  MV.MBWH..SNZ
Writing  features/9610-27-0523-04S.MVO_21_1.MV.MBWH..SNZ.pkl
Processing  MV.MBBE..BHZ
Writing  features/9610-27-0523-04S.MVO_21_1.MV.MBBE..BHZ.pkl
Processing  MV.MBBE..BHN
Writing  features/9610-27-0523-04S.MVO_21_1.MV.MBBE..B

Writing  features/9610-28-0805-50S.MVO_21_1.MV.MBWH..SHZ.pkl
Processing  MV.MBWH..SNZ
Writing  features/9610-28-0805-50S.MVO_21_1.MV.MBWH..SNZ.pkl
Processing  MV.MBBE..BHZ
Writing  features/9610-28-0805-50S.MVO_21_1.MV.MBBE..BHZ.pkl
Processing  MV.MBBE..BHN
Writing  features/9610-28-0805-50S.MVO_21_1.MV.MBBE..BHN.pkl
Processing  MV.MBBE..BHE
Writing  features/9610-28-0805-50S.MVO_21_1.MV.MBBE..BHE.pkl
Processing  MV.MBGB..BHZ
Writing  features/9610-28-0805-50S.MVO_21_1.MV.MBGB..BHZ.pkl
Processing  MV.MBGB..BHN
Writing  features/9610-28-0805-50S.MVO_21_1.MV.MBGB..BHN.pkl
Processing  MV.MBGB..BHE
Writing  features/9610-28-0805-50S.MVO_21_1.MV.MBGB..BHE.pkl
Reading  ./miniseed_c/MVOE_/1996/10/9610-28-1123-38S.MVO_21_1.mseed
Reading  /Users/thompsong/DATA/MVO/./miniseed_c/MVOE_/1996/10/9610-28-1123-38S.MVO_21_1.csv
Processing  MV.MBGA..BHZ
Writing  features/9610-28-1123-38S.MVO_21_1.MV.MBGA..BHZ.pkl
Processing  MV.MBGA..BHN
Writing  features/9610-28-1123-38S.MVO_21_1.MV.MBGA..BHN.pkl
Proce

...

Writing  features/9704-01-1839-12S.MVO_18_1.MV.MBGE..BHE.pkl
Processing  MV.MBGH..BHZ
Writing  features/9704-01-1839-12S.MVO_18_1.MV.MBGH..BHZ.pkl
Processing  MV.MBGH..BHN
Writing  features/9704-01-1839-12S.MVO_18_1.MV.MBGH..BHN.pkl
Processing  MV.MBGH..BHE
Writing  features/9704-01-1839-12S.MVO_18_1.MV.MBGH..BHE.pkl
Processing  MV.MBWH..SHZ
Writing  features/9704-01-1839-12S.MVO_18_1.MV.MBWH..SHZ.pkl
Processing  MV.MBBE..BHZ
Writing  features/9704-01-1839-12S.MVO_18_1.MV.MBBE..BHZ.pkl
Processing  MV.MBBE..BHN
Writing  features/9704-01-1839-12S.MVO_18_1.MV.MBBE..BHN.pkl
Processing  MV.MBBE..BHE
Writing  features/9704-01-1839-12S.MVO_18_1.MV.MBBE..BHE.pkl
Processing  MV.MBGB..BHZ
Writing  features/9704-01-1839-12S.MVO_18_1.MV.MBGB..BHZ.pkl
Processing  MV.MBGB..BHN
Writing  features/9704-01-1839-12S.MVO_18_1.MV.MBGB..BHN.pkl
Processing  MV.MBGB..BHE
Writing  features/9704-01-1839-12S.MVO_18_1.MV.MBGB..BHE.pkl
Reading  ./miniseed_c/MVOE_/1997/04/9704-01-1932-27S.MVO_18_1.mseed
Reading  /U

Writing  features/9704-02-1849-18S.MVO_18_1.MV.MBGH..BHN.pkl
Processing  MV.MBGH..BHE
Writing  features/9704-02-1849-18S.MVO_18_1.MV.MBGH..BHE.pkl
Processing  MV.MBWH..SHZ
Writing  features/9704-02-1849-18S.MVO_18_1.MV.MBWH..SHZ.pkl
Processing  MV.MBBE..BHZ
Writing  features/9704-02-1849-18S.MVO_18_1.MV.MBBE..BHZ.pkl
Processing  MV.MBBE..BHE
Writing  features/9704-02-1849-18S.MVO_18_1.MV.MBBE..BHE.pkl
Processing  MV.MBGB..BHZ
Writing  features/9704-02-1849-18S.MVO_18_1.MV.MBGB..BHZ.pkl
Processing  MV.MBGB..BHN
Writing  features/9704-02-1849-18S.MVO_18_1.MV.MBGB..BHN.pkl
Processing  MV.MBGB..BHE
Writing  features/9704-02-1849-18S.MVO_18_1.MV.MBGB..BHE.pkl
Reading  ./miniseed_c/MVOE_/1997/04/9704-02-2128-29S.MVO_18_1.mseed
Reading  /Users/thompsong/DATA/MVO/./miniseed_c/MVOE_/1997/04/9704-02-2128-29S.MVO_18_1.csv
Processing  MV.MBGA..BHZ
Writing  features/9704-02-2128-29S.MVO_18_1.MV.MBGA..BHZ.pkl
Processing  MV.MBGA..BHN
Writing  features/9704-02-2128-29S.MVO_18_1.MV.MBGA..BHN.pkl
Proce

Writing  features/9704-03-0043-24S.MVO_18_1.MV.MBWH..SHZ.pkl
Processing  MV.MBBE..BHZ
Writing  features/9704-03-0043-24S.MVO_18_1.MV.MBBE..BHZ.pkl
Processing  MV.MBBE..BHN
Writing  features/9704-03-0043-24S.MVO_18_1.MV.MBBE..BHN.pkl
Processing  MV.MBBE..BHE
Writing  features/9704-03-0043-24S.MVO_18_1.MV.MBBE..BHE.pkl
Processing  MV.MBGB..BHZ
Writing  features/9704-03-0043-24S.MVO_18_1.MV.MBGB..BHZ.pkl
Processing  MV.MBGB..BHN
Writing  features/9704-03-0043-24S.MVO_18_1.MV.MBGB..BHN.pkl
Processing  MV.MBGB..BHE
Writing  features/9704-03-0043-24S.MVO_18_1.MV.MBGB..BHE.pkl
Reading  ./miniseed_c/MVOE_/1997/04/9704-03-0126-41S.MVO_18_1.mseed
Reading  /Users/thompsong/DATA/MVO/./miniseed_c/MVOE_/1997/04/9704-03-0126-41S.MVO_18_1.csv
Processing  MV.MBGA..BHZ
Writing  features/9704-03-0126-41S.MVO_18_1.MV.MBGA..BHZ.pkl
Processing  MV.MBGA..BHN
Writing  features/9704-03-0126-41S.MVO_18_1.MV.MBGA..BHN.pkl
Processing  MV.MBGA..BHE
Writing  features/9704-03-0126-41S.MVO_18_1.MV.MBGA..BHE.pkl
Proce

Writing  features/9704-03-0627-42S.MVO_18_1.MV.MBBE..BHZ.pkl
Processing  MV.MBBE..BHN
Writing  features/9704-03-0627-42S.MVO_18_1.MV.MBBE..BHN.pkl
Processing  MV.MBBE..BHE
Writing  features/9704-03-0627-42S.MVO_18_1.MV.MBBE..BHE.pkl
Processing  MV.MBGB..BHZ
Writing  features/9704-03-0627-42S.MVO_18_1.MV.MBGB..BHZ.pkl
Processing  MV.MBGB..BHN
Writing  features/9704-03-0627-42S.MVO_18_1.MV.MBGB..BHN.pkl
Processing  MV.MBGB..BHE
Writing  features/9704-03-0627-42S.MVO_18_1.MV.MBGB..BHE.pkl
Reading  ./miniseed_c/MVOE_/1997/04/9704-04-0830-14S.MVO_18_1.mseed
Reading  /Users/thompsong/DATA/MVO/./miniseed_c/MVOE_/1997/04/9704-04-0830-14S.MVO_18_1.csv
Processing  MV.MBGA..BHZ
Writing  features/9704-04-0830-14S.MVO_18_1.MV.MBGA..BHZ.pkl
Processing  MV.MBGA..BHN
Writing  features/9704-04-0830-14S.MVO_18_1.MV.MBGA..BHN.pkl
Processing  MV.MBGA..BHE
Writing  features/9704-04-0830-14S.MVO_18_1.MV.MBGA..BHE.pkl
Processing  MV.MBLG..SHZ
Writing  features/9704-04-0830-14S.MVO_18_1.MV.MBLG..SHZ.pkl
Proce

FileNotFoundError: [Errno 2] No such file or directory: './WAV/MVOE_/1997/04/9704-08-1403-37S.MVO_18_1'